# Explore here

In [50]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from pickle import dump
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import (mean_squared_error, 
                             mean_absolute_error,
                             r2_score)
from sklearn.linear_model import LinearRegression, Lasso, Ridge


In [5]:
df_raw = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv", sep = ",")
df_raw.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2


In [6]:
df_raw.shape

(3140, 108)

In [9]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, fips to Urban_rural_code
dtypes: float64(61), int64(45), object(2)
memory usage: 2.6+ MB


In [ ]:
df['He']

In [16]:
df_interim = (
    df_raw
    .copy()
    .set_axis(
        df_raw.columns.str.replace(' ','_')
        .str.replace(r'\W','',regex=True)
        .str.lower()
        .str.slice(0,40), axis=1
    )
    .rename(columns={'heart_disease_number':'target'})
)

In [18]:
df = (
    df_interim
    .copy()
    .reindex(
        columns=(
            ['target'] + 
            [c for c in df_interim.columns.to_list() if c not in ['target']]
        )
    )
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, target to urban_rural_code
dtypes: float64(61), int64(45), object(2)
memory usage: 2.6+ MB


In [24]:
data_types = df.dtypes
numeric_col = [i for i in list(data_types[data_types != "object"].index) if i != "target"]

scaler = StandardScaler()
norm_features = scaler.fit_transform(df[numeric_col])

# Crear un nuevo DataFrame con las variables numéricas escaladas
df_scal = pd.DataFrame(norm_features, index=df.index, columns=numeric_col)
df_scal["target"] = df["target"]
df_scal.head()


,fips,tot_pop,09,09_yo__of_total_pop,19oct,1019_yo__of_total_pop,2029,2029_yo__of_total_pop,3039,3039_yo__of_total_pop,...,diabetes_prevalence,diabetes_lower_95_ci,diabetes_upper_95_ci,diabetes_number,ckd_prevalence,ckd_lower_95_ci,ckd_upper_95_ci,ckd_number,urban_rural_code,target
0,-1.940874,-0.145679,-0.142421,0.158006,-0.135556,0.573496,-0.153144,0.027610,-0.139384,0.588469,...,-0.063696,-0.071720,-0.089834,-0.129902,-0.609615,-0.582796,-0.669652,-0.147523,-1.082865,3345
1,-1.940742,0.341296,0.287476,-0.242861,0.320383,-0.193107,0.183774,-0.469965,0.230620,-0.110300,...,-0.394103,-0.414900,-0.337677,0.376251,-0.433549,-0.393279,-0.343373,0.389791,-0.420704,13414
2,-1.940610,-0.237785,-0.239429,-0.419441,-0.246181,-0.439718,-0.225971,0.272104,-0.218759,0.656538,...,2.432709,2.483064,2.317776,-0.183415,1.855312,1.880929,1.777443,-0.204321,0.903618,2159
3,-1.940478,-0.245223,-0.246032,-0.426966,-0.254791,-0.609076,-0.230792,0.396168,-0.220555,1.264959,...,0.376846,0.423984,0.299632,-0.229096,-0.257483,-0.203761,-0.180233,-0.242100,-1.745026,1533
4,-1.940346,-0.138966,-0.135053,0.186249,-0.137140,0.216679,-0.155888,-0.200808,-0.143570,0.088582,...,0.156575,0.195197,0.158008,-0.111247,-0.081417,-0.014244,-0.017093,-0.124105,-1.745026,4101


In [27]:
X = df_scal.drop(columns=["target"])
y = df_scal["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2024)
train_indices = list(X_train.index)
test_indices = list(X_test.index)


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2024)
k = int(len(X_train.columns) * 0.3)  # Seleccionar el 30% de las mejores características
selection_model = SelectKBest(score_func=f_regression, k=k)
selection_model.fit(X_train, y_train)
ix = selection_model.get_support()

X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns=X_train.columns[ix])
X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns=X_test.columns[ix])

X_train_sel.head()


,tot_pop,09,19oct,2029,3039,4049,5059,6069,7079,80,...,family_medicinegeneral_practice_primary_,total_specialist_physicians_2019,total_population,population_aged_60,county_pop2018_18_and_older,anycondition_number,obesity_number,copd_number,diabetes_number,ckd_number
0,-0.281868,-0.269740,-0.285588,-0.271926,-0.262468,-0.274546,-0.285927,-0.302935,-0.311833,-0.281842,...,-0.283113,-0.281520,-0.279999,-0.298667,-0.283179,-0.301387,-0.302332,-0.335172,-0.285063,-0.287253
1,-0.275041,-0.265960,-0.276623,-0.263937,-0.254118,-0.263526,-0.278552,-0.298618,-0.305030,-0.293946,...,-0.298656,-0.282222,-0.272793,-0.298357,-0.276250,-0.285892,-0.289438,-0.308769,-0.264827,-0.273280
2,-0.262205,-0.267084,-0.276954,-0.259943,-0.241767,-0.247881,-0.260876,-0.263935,-0.274317,-0.270401,...,-0.263607,-0.274765,-0.262614,-0.272753,-0.259301,-0.290979,-0.293911,-0.311250,-0.266138,-0.263835
3,-0.216081,-0.205315,-0.209093,-0.221109,-0.208653,-0.218432,-0.221147,-0.224443,-0.219446,-0.193050,...,-0.184939,-0.230716,-0.209431,-0.205404,-0.217928,-0.218076,-0.210236,-0.225680,-0.197802,-0.204968
4,-0.263695,-0.256774,-0.262266,-0.241948,-0.247646,-0.261869,-0.270452,-0.284623,-0.301973,-0.258877,...,-0.238906,-0.272659,-0.261603,-0.284334,-0.263599,-0.282848,-0.280896,-0.319012,-0.269533,-0.271857


In [29]:
X_test_sel.sample(5,random_state=2024)

,tot_pop,09,19oct,2029,3039,4049,5059,6069,7079,80,...,family_medicinegeneral_practice_primary_,total_specialist_physicians_2019,total_population,population_aged_60,county_pop2018_18_and_older,anycondition_number,obesity_number,copd_number,diabetes_number,ckd_number
27,-0.269347,-0.270577,-0.282560,-0.265793,-0.258140,-0.267345,-0.272208,-0.261936,-0.263224,-0.259706,...,-0.283180,-0.256668,-0.267192,-0.267785,-0.267313,-0.273310,-0.278721,-0.277658,-0.247752,-0.257884
140,-0.268792,-0.265266,-0.271230,-0.262364,-0.250382,-0.259914,-0.273987,-0.285521,-0.290339,-0.255478,...,-0.258305,-0.273516,-0.266205,-0.277314,-0.269062,-0.285659,-0.283945,-0.317103,-0.271617,-0.272892
55,-0.249565,-0.256726,-0.262857,-0.251369,-0.243379,-0.248111,-0.246104,-0.235569,-0.234088,-0.235166,...,-0.238281,-0.229812,-0.247453,-0.236985,-0.246608,-0.244596,-0.248447,-0.229242,-0.228760,-0.227738
1,-0.233956,-0.229717,-0.241758,-0.237994,-0.223066,-0.232052,-0.238308,-0.223400,-0.233201,-0.230358,...,-0.239170,-0.246536,-0.232861,-0.228418,-0.233724,-0.261416,-0.271253,-0.263979,-0.235483,-0.223340
428,-0.259657,-0.267324,-0.269503,-0.254617,-0.246278,-0.244131,-0.246970,-0.270918,-0.277718,-0.265841,...,-0.275223,-0.263503,-0.256255,-0.271776,-0.256988,-0.265694,-0.265854,-0.288346,-0.239852,-0.249086


In [30]:
X_train_sel["target"] = list(y_train)
X_test_sel["target"] = list(y_test)

X_train_sel.to_csv("../data/processed/clean_train.csv", index=False)
X_test_sel.to_csv("../data/processed/clean_test.csv", index=False)


In [31]:
df = pd.concat([X_train_sel, X_test_sel])
df.sample(5,random_state=2024)

,tot_pop,09,19oct,2029,3039,4049,5059,6069,7079,80,...,total_specialist_physicians_2019,total_population,population_aged_60,county_pop2018_18_and_older,anycondition_number,obesity_number,copd_number,diabetes_number,ckd_number,target
2190,-0.268145,-0.277036,-0.284027,-0.254920,-0.242522,-0.245212,-0.263709,-0.289491,-0.292163,-0.289387,...,-0.267637,-0.267451,-0.292380,-0.264280,-0.268991,-0.272636,-0.270469,-0.250407,-0.260342,1293
284,-0.296682,-0.289764,-0.300276,-0.280802,-0.273226,-0.282299,-0.301660,-0.324666,-0.336532,-0.303066,...,-0.296733,-0.294868,-0.322642,-0.296960,-0.319066,-0.320142,-0.352349,-0.297836,-0.302520,340
2,-0.189822,-0.184478,-0.186670,-0.195610,-0.183767,-0.174788,-0.191157,-0.202770,-0.190508,-0.197776,...,-0.195837,-0.188882,-0.194914,-0.191578,-0.169821,-0.191111,-0.113706,-0.152255,-0.166283,3235
2476,-0.309676,-0.302156,-0.313049,-0.290345,-0.281556,-0.294332,-0.316549,-0.341819,-0.351519,-0.331254,...,-0.306228,-0.308640,-0.343148,-0.310013,-0.334400,-0.336272,-0.367109,-0.310609,-0.316105,61
1479,-0.253439,-0.252491,-0.257062,-0.243381,-0.242297,-0.251746,-0.252799,-0.265413,-0.271014,-0.230192,...,-0.261303,-0.250011,-0.257410,-0.252426,-0.265866,-0.262751,-0.293881,-0.252794,-0.252709,1300


In [32]:
X_test_sel.head()

,tot_pop,09,19oct,2029,3039,4049,5059,6069,7079,80,...,total_specialist_physicians_2019,total_population,population_aged_60,county_pop2018_18_and_older,anycondition_number,obesity_number,copd_number,diabetes_number,ckd_number,target
0,-0.197710,-0.178593,-0.192087,-0.176182,-0.184992,-0.201246,-0.212742,-0.229339,-0.218313,-0.197610,...,-0.224905,-0.194733,-0.212700,-0.200499,-0.193974,-0.196201,-0.211810,-0.197701,-0.199404,2424
1,-0.233956,-0.229717,-0.241758,-0.237994,-0.223066,-0.232052,-0.238308,-0.223400,-0.233201,-0.230358,...,-0.246536,-0.232861,-0.228418,-0.233724,-0.261416,-0.271253,-0.263979,-0.235483,-0.223340,1677
2,-0.189822,-0.184478,-0.186670,-0.195610,-0.183767,-0.174788,-0.191157,-0.202770,-0.190508,-0.197776,...,-0.195837,-0.188882,-0.194914,-0.191578,-0.169821,-0.191111,-0.113706,-0.152255,-0.166283,3235
3,-0.276147,-0.272587,-0.280526,-0.264643,-0.257262,-0.261777,-0.281970,-0.294098,-0.301185,-0.285822,...,-0.274973,-0.274971,-0.290018,-0.275964,-0.285265,-0.290311,-0.292863,-0.264155,-0.271986,1013
4,-0.275230,-0.269190,-0.279769,-0.261920,-0.257282,-0.261846,-0.281057,-0.293982,-0.301382,-0.288972,...,-0.282354,-0.273503,-0.292363,-0.275453,-0.281998,-0.286269,-0.307624,-0.258205,-0.270563,914


In [33]:
df_train = pd.read_csv("../data/processed/clean_train.csv")
df_test = pd.read_csv("../data/processed/clean_test.csv")

df_train.head()

,tot_pop,09,19oct,2029,3039,4049,5059,6069,7079,80,...,total_specialist_physicians_2019,total_population,population_aged_60,county_pop2018_18_and_older,anycondition_number,obesity_number,copd_number,diabetes_number,ckd_number,target
0,-0.281868,-0.269740,-0.285588,-0.271926,-0.262468,-0.274546,-0.285927,-0.302935,-0.311833,-0.281842,...,-0.281520,-0.279999,-0.298667,-0.283179,-0.301387,-0.302332,-0.335172,-0.285063,-0.287253,622
1,-0.275041,-0.265960,-0.276623,-0.263937,-0.254118,-0.263526,-0.278552,-0.298618,-0.305030,-0.293946,...,-0.282222,-0.272793,-0.298357,-0.276250,-0.285892,-0.289438,-0.308769,-0.264827,-0.273280,881
2,-0.262205,-0.267084,-0.276954,-0.259943,-0.241767,-0.247881,-0.260876,-0.263935,-0.274317,-0.270401,...,-0.274765,-0.262614,-0.272753,-0.259301,-0.290979,-0.293911,-0.311250,-0.266138,-0.263835,1030
3,-0.216081,-0.205315,-0.209093,-0.221109,-0.208653,-0.218432,-0.221147,-0.224443,-0.219446,-0.193050,...,-0.230716,-0.209431,-0.205404,-0.217928,-0.218076,-0.210236,-0.225680,-0.197802,-0.204968,2176
4,-0.263695,-0.256774,-0.262266,-0.241948,-0.247646,-0.261869,-0.270452,-0.284623,-0.301973,-0.258877,...,-0.272659,-0.261603,-0.284334,-0.263599,-0.282848,-0.280896,-0.319012,-0.269533,-0.271857,924


In [34]:
X_train = df_train.drop(["target"], axis = 1)
y_train = df_train["target"]
X_test = df_test.drop(["target"], axis = 1)
y_test = df_test["target"]

In [36]:
model = LogisticRegression()
model.fit(X_train, y_train)

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [37]:
print(f"Intercep (a): {model.intercept_}")
print(f"Coefficients: {model.coef_}")

Intercep (a): [-0.34167975 -0.34205296 -0.33819209 ... -2.2113287  -2.30620629
 -2.25039114]
Coefficients: [[-0.07262053 -0.06770616 -0.07461278 ... -0.12098927 -0.08452989
  -0.08508305]
 [-0.07276516 -0.06838146 -0.07527683 ... -0.12098556 -0.08439072
  -0.08473013]
 [-0.07221995 -0.06745512 -0.07416271 ... -0.12058771 -0.08418167
  -0.08457014]
 ...
 [ 0.23923108  0.42199183  0.26823441 ...  0.04900048  0.57295239
   0.13006238]
 [ 0.20224323  0.170112    0.12123989 ...  0.25389167  0.11925277
   0.31832908]
 [ 0.23771227  0.14389492  0.15407311 ...  0.06454295  0.39654263
   0.29437124]]


In [43]:
reg_lasso = Lasso(alpha=5)
reg_lasso.fit(X_train,y_train)
y_pred = reg_lasso.predict(X_test) 

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.096e+08, tolerance: 6.594e+07
  model = cd_fast.enet_coordinate_descent(


In [45]:
reg_mlin = LinearRegression()
reg_mlin.fit(X_train, y_train)
y_pred_m = reg_mlin.predict(X_test)

In [48]:
print(f'MSE: {mean_squared_error(y_test, y_pred_m)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_m))}')

print(f'MAE: {mean_absolute_error(y_test, y_pred_m)}')
print(f'R2: {r2_score(y_test, y_pred_m)}')

MSE: 376889.5748633163
RMSE: 613.9133284620202
MAE: 198.16845734028902
R2: 0.997762855609049


In [ ]:
# Optimizacion del modelo
alpha = 1.0
lasso_model = Lasso(alpha = alpha)

lasso_model.fit(X_train, y_train)

score = lasso_model.score(X_test, y_test)
print("Coefficients:", lasso_model.coef_)
print("R2 score:", score)

Coefficients: [ 4278.056945    1893.84991505 -2384.1104407   -834.31828085
  -907.2738787   3653.3544555   1376.07678464 -1346.72798971
  3933.820522     925.01004945  1029.11099189 -5367.08751509
   -20.87463036  -829.60171494  -295.03666088  2570.91391627
   425.68659162    29.5404282  -1629.47508152   123.84750622
   464.47545783  -343.98993849  -880.06955142 -2877.2614723
  3094.76109344   150.85642036  1938.48632696  2939.69691358
  5307.11517542   505.79666908 -1170.88501683]
R2 score: 0.9987523072659658


/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.192e+08, tolerance: 6.594e+07
  model = cd_fast.enet_coordinate_descent(


In [51]:
dump(lasso_model, open("../models/lasso_alpha-1.0.sav", "wb"))